In [1]:
#doc: https://owlready2.readthedocs.io/en/v0.31/

from owlready2 import *
#onto = get_ontology("file://BVCO_inferred.owl")
onto = get_ontology("file://BattINFO_inferred.owl")
onto.load()

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


get_ontology("http://emmo.info/BattINFO#")

In [63]:
import mwclient
import getpass
from pprint import pprint
site = mwclient.Site('onto-wiki.eu', path='/w/')
user = input("Enter wiki username")
password = getpass.getpass("Enter wiki password")
site.login(user, password)

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Enter wiki username Simon Stier
Enter wiki password ·········


In [66]:
page = site.pages["TestPage/DisplayTitle"]
page.edit(page.text() + "\r\nTest", 'Test Edit')

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


OrderedDict([('result', 'Success'),
             ('pageid', 617),
             ('title', 'TestPage/DisplayTitle'),
             ('contentmodel', 'wikitext'),
             ('oldrevid', 1236),
             ('newrevid', 1237),
             ('newtimestamp', '2021-05-20T06:50:16Z')])

In [53]:
import mwparserfromhell

def mergeWikiText(text1, text2):
    if (text1 == text2): return text1
    if (text1 == ""): return text2
    if (text2 == ""): return text1
    code1 = mwparserfromhell.parse(text1)
    code2 = mwparserfromhell.parse(text2)
    r1 = []
    r2 = []
    for template in code1.filter_templates(recursive=True):
        if template.name.matches("Ontology Class"): t1 = template
        if template.name.matches("Ontology Relation"): r1.append(template)
    for template in code2.filter_templates(recursive=True):
        if template.name.matches("Ontology Class"): t2 = template
        if template.name.matches("Ontology Relation"): r2.append(template)
    #print("T1: " + str(t1))
    #print("T2: " + str(t2))
    for p in t2.params:
        if (not t1.has(p.name)):
            t1.add(p.name, p.value)
    t1.remove("relations") 
    for rel2 in r2:
        exists = False
        for rel1 in r1:
            allParamsEqual = True
            for p in rel2.params:
                if (rel1.has(p.name)):
                    if (not rel1.get(p.name).value.matches(rel2.get(p.name).value)): allParamsEqual = False
                    #print("Compare {}:{} with :{} -> res={}".format(p.name, rel1.get(p.name).value,rel2.get(p.name).value, 
                    #                                                rel1.get(p.name).value.matches(rel2.get(p.name).value)))
                else: allParamsEqual = False
            if (allParamsEqual): exists = True
        if not exists: r1.append(rel2)
    r1string = ""
    for rel1 in r1:
        r1string += "\r\n   " + str(rel1)
    t1.add("relations", r1string)
    #print("Tmerged: " + str(t1))
    return str(t1)

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [67]:
counter = 0
for c in onto.classes():
    #try:
    if (True):#c.iri == "http://emmo.info/emmo#EMMO_f8bd64d5_5d3e_4ad4_a46e_c30714fecb7f"):
        wikitext = "{{Ontology Class"
        wikitext += "\r\n|iri=" + c.iri
        c_name = c.iri.split('#')[-1];
        c_label = c_name;
        if (hasattr(c, 'prefLabel') and len(c.prefLabel) > 0): c_label = c.prefLabel[0]
        elif (hasattr(c, 'label') and len(c.label) > 0): c_label = c.label[0]
        print("Class '{}' ({})".format(c_label, c_name))
        wikitext += "\r\n|label=" + c_label
        print("   Comments:",c.comment)
        print("   Labels:",c.label)
        
        wikitext += "\r\n|relations="
        print("   Subclasses:")
        for subc in c.subclasses():
            subc_name = str(subc);
            if (hasattr(subc, 'iri')): subc_name = subc.iri.split('#')[-1]
            if (not hasattr(subc, 'iri')): continue
            subc_label = subc_name;
            if (hasattr(subc, 'prefLabel') and len(subc.prefLabel) > 0): subc_label = subc.prefLabel[0]
            elif (hasattr(subc, 'label') and len(subc.label) > 0): subc_label = subc.label[0]
            else: continue
            wikitext += "\r\n   {{Ontology Relation"
            wikitext += "\r\n   |property=Has subclass"
            wikitext += "\r\n   |object=" + subc_name
            wikitext += "\r\n   }}"
            print("      Class '{}' ({})".format(subc_label, subc_name))
            
        print("   Superclasses:")
        for supc in c.is_a:
            supc_name = str(supc);
            if (hasattr(supc, 'iri')): supc_name = supc.iri.split('#')[-1]
            if (not hasattr(subc, 'iri')): continue
            supc_label = supc_name;
            if (hasattr(supc, 'prefLabel') and len(supc.prefLabel) > 0): supc_label = supc.prefLabel[0]
            elif (hasattr(supc, 'label') and len(supc.label) > 0): supc_label = supc.label[0]
            else: continue
            wikitext += "\r\n   {{Ontology Relation"
            wikitext += "\r\n   |property=Is a"
            wikitext += "\r\n   |object=" + supc_name
            wikitext += "\r\n   }}"
            print("      Class '{}' ({})".format(supc_label, supc_name))
            
        for p in onto.properties():
            p_name = p.iri.split('#')[-1]
            p_label = p_name
            if (hasattr(p, 'prefLabel') and len(p.prefLabel) > 0): p_label = p.prefLabel[0]
            elif (hasattr(p, 'label') and len(p.label) > 0): p_label = p.label[0]
            if (hasattr(c, p_name)):
                objects = getattr(c, p_name)
                #print(p_name, type(objects), str(objects)) object may be of type <class 'int'> for prop hasNumericalData (EMMO_faf79f53_749d_40b2_807c_d34244c192f4)
                if (objects is not None and hasattr(objects, 'len') and len(objects) > 0):
                    for o in objects:
                        o_name = str(o)
                        if (hasattr(o, 'iri')): o_name = o.iri.split('#')[-1]
                        o_label = o_name
                        if (hasattr(o, 'prefLabel') and len(o.prefLabel) > 0): o_label = o.prefLabel[0]
                        elif (hasattr(o, 'label') and len(o.label) > 0): o_label = o.label[0]
                        wikitext += "\r\n   {{Ontology Relation"
                        wikitext += "\r\n   |property=" + p_label
                        wikitext += "\r\n   |object=" + o_name.replace("|","").replace("{","").replace("}","")
                        wikitext += "\r\n   }}"
                        print("   Relation: {} ({}) {} ({})".format(p_label, p_name, o_label, o_name))
        wikitext += "\r\n}}"
        page = site.pages[c_name]
        wikitext = mergeWikiText(page.text(),wikitext)
        if (not wikitext == page.text()): page.edit(wikitext, 'Updated from BattINFO (Autoimport from owl)')
        #print(wikitext)
        counter += 1
        if (counter == 99999999): break;
            #if (len(c[p_name]) > 0):
            #try:
            #    print(c.p_name)
            #    print("   Property:", p_label, p_name )
            #except:
            #    i = 0
            #print("Relation: ", c.p_name)
        #for prop in c.get_properties(c):
            #if (hasattr(prop, 'prefLabel') and len(prop.prefLabel) > 0): print("   Properties:",prop.prefLabel[0])
            #if (hasattr(prop, 'label') and len(prop.label) > 0): print("   Properties:",prop.label[0])
            #if (hasattr(prop, 'rdf-schema.label') and len(prop.rdf-schema.label) > 0): print("   Properties:",prop.rdf-schema.label[0])   
            #elif: 
            #print("   Properties:",prop, type(prop))
    #except:
    #    print("   Error")
print(counter)


/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Class 'Arrangement' (EMMO_25a3da5e_eab1_42dd_8081_61dd09d34e1b)
   Comments: []
   Labels: []
   Subclasses:
      Class 'Array' (EMMO_28fbea28_2204_4613_87ff_6d877b855fcd)
   Superclasses:
      Class 'State' (EMMO_36c79456_e29c_400d_8bd3_0eedddb82652)
      Class 'Ordered' (EMMO_c03bab53_fed3_4142_9741_cc7fc806f0a6)
Class 'SpatialOrderedElement' (EMMO_42fc460a_4bf3_4d0b_8dee_3c7efcefebb5)
   Comments: []
   Labels: []
   Subclasses:
   Superclasses:
      Class 'OrderedElement' (EMMO_2e9ace8a_1155_45b5_a066_d5fd9774e76c)
Class 'QuantitativeProperty' (EMMO_dd4a7f3e_ef56_466c_ac1a_d2716b5f87ec)
   Comments: ['A quantitative property is always expresssed as a quantity (i.e. a number and a reference unit). For the EMMO, a nominalistic ontology, there is no property as abstract object.\n\nA property is a sign that stands for an object according to a specific code shared by some observers.\n\nFor quantititative properties, one possible code that is shared between the scientific community (

ReadTimeout: HTTPSConnectionPool(host='onto-wiki.eu', port=443): Read timed out. (read timeout=30)

In [5]:
for p in onto.properties():
    p_name = p.iri.split('#')[-1]
    if (p_name == p.iri): p_name = p.iri.split('/')[-1]
    p_label = p_name
    if (hasattr(p, 'prefLabel') and len(p.prefLabel) > 0): p_label = p.prefLabel[0]
    elif (hasattr(p, 'label') and len(p.label) > 0): p_label = p.label[0]
    print("Property: {} ({}) {}".format(p_label, p_name, p.iri))
    #print("   Comments:",c.comment)
    #print("   Labels:",c.label)

Property: hasSymbolData (EMMO_23b579e1_8088_45b5_9975_064014026c42) http://emmo.info/emmo#EMMO_23b579e1_8088_45b5_9975_064014026c42
Property: hasNumericalData (EMMO_faf79f53_749d_40b2_807c_d34244c192f4) http://emmo.info/emmo#EMMO_faf79f53_749d_40b2_807c_d34244c192f4
Property: mereotopological (EMMO_03212fd7_abfd_4828_9c8e_62c293052d4b) http://emmo.info/emmo#EMMO_03212fd7_abfd_4828_9c8e_62c293052d4b
Property: EMMORelation (EMMO_ec2472ae_cf4a_46a5_8555_1556f5a6c3c5) http://emmo.info/emmo#EMMO_ec2472ae_cf4a_46a5_8555_1556f5a6c3c5
Property: hasSpatialLast (EMMO_050b47b6_2583_43f7_95eb_a216b41cd5f6) http://emmo.info/emmo#EMMO_050b47b6_2583_43f7_95eb_a216b41cd5f6
Property: hasSpatialDirectPart (EMMO_b2282816_b7a3_44c6_b2cb_3feff1ceb7fe) http://emmo.info/emmo#EMMO_b2282816_b7a3_44c6_b2cb_3feff1ceb7fe
Property: hasQuantitativeProperty (EMMO_0aa934ee_1ad4_4345_8a7f_bc73ec67c7e5) http://emmo.info/emmo#EMMO_0aa934ee_1ad4_4345_8a7f_bc73ec67c7e5
Property: hasProperty (EMMO_e1097637_70d2_4895_973f_2